In [17]:
import pandas as pd
import string
import re


In [18]:
# Load the data
df = pd.read_csv('spotify_millsongdata.csv')
df.describe()

,artist,song,link,text
count,57650,57650,57650,57650
unique,643,44824,57650,57494
top,Donna Summer,Have Yourself A Merry Little Christmas,/a/abba/ahes+my+kind+of+girl_20598417.html,I just came back from a lovely trip along the ...
freq,191,35,1,6


In [19]:
def preprocess(text):
    # Remove integers
    text = re.sub(r'\d+', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation marks
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    return text

df['text'] = df['text'].apply(preprocess)

# Identify the rows that contain duplicated text in the 'song' column
no_covers = ~df['text'].duplicated() 

# Filter the DataFrame to include only the rows with unique text
df = df[no_covers]
df.describe()

,artist,song,link,text
count,57431,57431,57431,57431
unique,643,44768,57431,57431
top,Donna Summer,Have Yourself A Merry Little Christmas,/a/abba/ahes+my+kind+of+girl_20598417.html,look at her face its a wonderful face \r\nand...
freq,190,31,1,1


In [20]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import math
import numpy as np

#NAIVE BAYES
corpus = df['text']


#Counting the occurences of each word in the whole data set
#We only consider words that have a minimum frequency of occurrence of 10 percent
vectorizer = CountVectorizer(min_df = 0.1)

X = vectorizer.fit_transform(corpus).toarray()
vectorizer.get_feature_names_out()
#print(X)
"""
vectorizer_bigram = CountVectorizer(analyzer='word', min_df = 0.1, ngram_range=(2, 2))
X_bigram = vectorizer_bigram.fit_transform(corpus).toarray()
"""
y = df['artist']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = MultinomialNB()
clf.fit(X_train,y_train)
score = clf.score(X_test,y_test)
print("Accuracy on test set is:", score)





KeyboardInterrupt: 

In [26]:
#Feature: average or max length of sentences
from statistics import mean
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import scale


averages = []
maxima = []
for song in df['text']:
    length_of_sentences = []
    arr = song.splitlines() #Making a list of sentences for each song
    for sentence in arr:
        length_of_sentences.append(len(sentence.split()))
    averages.append(mean(length_of_sentences))
    maxima.append(max(length_of_sentences))
#Reshaping the feature lists for the model

#print(maxima[100:120])
#print(averages[100:120])
X_average = scale(np.array(averages)).reshape(-1, 1)
X_maxima = scale(np.array(maxima)).reshape(-1, 1)

#Maximum and average length per song both as feature
X_merged= np.column_stack((X_average, X_maxima))
X_merged = X_merged.reshape(-1,2)

y = df['artist']
X_train, X_test, y_train, y_test = train_test_split(X_merged, y, test_size=0.2, random_state=42)

clf2 = LogisticRegression(random_state=1, solver = 'liblinear', max_iter=200, penalty = 'l1')
clf2.fit(X_train, y_train)
print( "Accuracy on test set is:", clf2.score(X_test,y_test))





Accuracy on test set is: 0.00957604248280665
